In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# For future iterations of the project to ensure manual list of docs is not needed

# import glob

## Specify the directory path and file format
# directory_path = './'
# file_format = '*.xlsx'  # For spreadsheets

## Use glob.glob to find files
# files_list = glob.glob(directory_path + file_format)

## Check list length to confirm all files are present
# len(files_list)

In [ ]:
docs = []  # list for storing files

In [ ]:
# Concatenate multiple spreadsheets
# Here we are using 1221-1243 data obtained on Feb 1 2024
dataframes = [] 

for doc in docs:
    repo = pd.read_excel(doc, header=1, dtype = str)  
    dataframes.append(repo)  

combined_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
# Checking data types to ensure all is string
combined_df.info()

In [ ]:
# Get rid of duplicated student records
combined_df = combined_df.drop_duplicates(subset='ID', keep='first')

In [ ]:
# Confirm datatypes again
combined_df.info()

In [ ]:
# Create a mask for rows where 'Item Description' contains 'Amazon'
mask_amazon = combined_df['Item Description'].str.contains('Amazon', case=False, na=False)

# Create a mask for rows where 'Item Description' contains 'Boeing'
mask_boeing = combined_df['Item Description'].str.contains('Boeing', case=False, na=False)

# Create a mask for rows where 'Item Description' contains 'Boeing Spouses Waiver'
mask_boeing_spouses_waiver = combined_df['Item Description'].str.contains('Boeing Spouses Waiver', case=False, na=False)

# Filter the records. We are dropping the spouses here 
filtered_df = combined_df[(mask_amazon | (mask_boeing & ~mask_boeing_spouses_waiver))]

filtered_df

In [ ]:
# Confirm unique values for Item description to ensure that no unrelevant items are present
unique_waivers = filtered_df['Item Description'].unique()
unique_waivers

In [ ]:
pd.set_option('display.max_rows', None)    # didplays entire dataframe

In [ ]:
# Sort by Term (most recent first) and Item Description (ABC ascending)
sorted_df = filtered_df.sort_values(by=['Item Term', 'Item Description'], ascending=[False, True])

In [ ]:
len(sorted_df)    # confirm dataframe size 

In [ ]:
sorted_df.columns      # confirm columns

In [ ]:
# Get rid of irrelevant columns
columns_to_remove = ['Business Unit', 'Item Type', 'Item Amt', 'Subject', 'Catalog', 'Class Title']

sorted_df = sorted_df.drop(columns=columns_to_remove)

In [ ]:
# Confirm unique program values
unique_programs = sorted_df['Prim Prog'].unique()
unique_programs

In [ ]:
# Add new column School which will reassign programs among SBM/STC/SOEL/SHSS
# The lists are not exhaustive and contain only options available on this report. Update accordingly upon next iteration!
SBM_progs = ['MSTPM', 'AS-B', 'BSPM', 'BAMGT', 'UC-BU', 'MBA', 'BSBA', 'BSDA', 'DBA', 'BSIT', 'MSGSC', 'GCRTB', 'MSPM', 'MSMG', 'MSML', 'MSHA', 'UND-B']
STC_progs = ['BSIS', 'BSDS', 'MSDS', 'BSCY', 'DIT', 'BSIT', 'BSACS', 'MSCSI', 'MSCY', 'BSAPC', 'MSISE', 'BSCYB']
SOEL_progs = ['GCRTT', 'UC-TC', 'EDD', 'UND-T', 'BAED']
SHSS_progs = ['MA', 'BAAP', 'AS', 'BSGS']

def map_program_to_school(program):
    if program in SBM_progs:
        return 'SBM'
    elif program in STC_progs:
        return 'STC'
    elif program in SOEL_progs:
        return 'SOEL'
    elif program in SHSS_progs:
        return 'SHSS'
    else:
        return 'Other'  # For programs not listed

sorted_df['School'] = sorted_df['Prim Prog'].apply(map_program_to_school)


In [ ]:
# Get contact information and advisor name from a different query
adv_data = pd.read_excel("advisor.xlsx", header=1, dtype = str)

In [ ]:
# Check datatypes
adv_data.info()

In [ ]:
# Left merge based on ID
merged_df = pd.merge(sorted_df, adv_data[['ID', 'First Enrollment Term', 'Advisor', 'Email', 'Phone']], on='ID', how='left')

In [ ]:
# Filter and display records where 'First Enrollment Term' is NaN
nan_records = merged_df[pd.isna(merged_df['First Enrollment Term'])]

len(nan_records)      # check the number of records 

In [ ]:
# Review the dataframe format 
merged_df.head()

In [ ]:
# This table will allow us to identify students who have already graduated 
alums_data = pd.read_excel("contact.xlsx", header = 1, dtype = str)

In [ ]:
alums_data.info()

In [ ]:
alums_data.head()

In [ ]:
columns_to_add = ['Compl Term', 'Preferred Email', 'Home Email', 'Work Email']

# Merge with specific columns
final_df = pd.merge(merged_df, alums_data[['ID'] + columns_to_add], on='ID', how='left')


In [ ]:
# Create 'Alumni?' column based on whether 'Compl Term' is not null
final_df['Alumni?'] = np.where(final_df['Compl Term'].notnull(), 'Yes', 'No')

In [ ]:
final_df.head()

In [ ]:
# Function to merge existing 'Email' with 'Preferred Email', 'Home Email', and 'Work Email', remove duplicates, and NaN values
def merge_emails(row):
    emails = [row['Email'], row['Preferred Email'], row['Home Email'], row['Work Email']]
    unique_emails = list(set([email for email in emails if pd.notnull(email) and email != '']))
    return ', '.join(unique_emails)

final_df['Combined Email'] = final_df.apply(merge_emails, axis=1)

In [ ]:
final_df.head()

In [ ]:
final_df.columns

In [ ]:
# Change column order 
columns_ordered = ['Item Term', 'First Enrollment Term', 'ID', 'Name', 'School', 'Prim Prog', 'Item Description', 'Alumni?', 'Compl Term', 'Advisor', 'Combined Email', 'Phone']
df_reordered = final_df[columns_ordered]

In [ ]:
df_reordered.head()

In [ ]:
# Get records where both 'First Enrollment Term' and 'Compl Term' are NaN - needs manual lookup!
condition = pd.isna(df_reordered['First Enrollment Term']) & pd.isna(df_reordered['Compl Term'])

# Filter the DataFrame based on the condition
rows_with_both_nan = df_reordered[condition]

In [ ]:
len(rows_with_both_nan)
# rows_with_both_nan.to_excel("worksheet.xlsx", index=False)     - use to save for manual lookup in Peoplesoft 

In [ ]:
# Exclude rows_with_both_nan from the original df
df_reordered = df_reordered[~condition]

In [ ]:
rows_with_both_nan.head()

In [ ]:
# Once manual lookup is complete, read the file back into the notebook
manual_entries = pd.read_excel("worksheet.xlsx")

In [ ]:
# Concatenate records 
updated_df = pd.concat([df_reordered, manual_entries], ignore_index=True, sort = False)

In [ ]:
# Get rid of Item Term since we no longer need it for reference
updated_df = updated_df.drop(['Item Term'], axis = 1)

In [ ]:
updated_df.head()

In [ ]:
# Get all records that have "Discontinued" in Notes to avoid students who are no longer active
mask = updated_df['Notes'].str.strip().str.contains('Discontinued', case=False, na=False)

discontinued_df = updated_df[mask]

discontinued_df

In [ ]:
len(discontinued_df)     # checking number of discontinued students

In [ ]:
discontinued_df.to_excel("discontinued.xlsx", index=False)    # save to a separate spreadsheet for reference 

In [ ]:
# Get an updated dataframe without discontinued students
mask = ~updated_df['Notes'].str.strip().str.contains('Discontinued', case=False, na=False)

updated_df = updated_df[mask]

In [ ]:
updated_df.head()

In [ ]:
# Check for duplicated records 
duplicates = updated_df.duplicated(subset='ID', keep=False)
updated_df[duplicates]

In [ ]:
# Get rid of duplicates 
df_cleaned = updated_df.drop_duplicates(subset='ID', keep='first')

In [ ]:
len(df_cleaned)  # confirm total number of unique students 

In [ ]:
# Status Applied for Graduation; timeframe = from January 1 2020 to February 1 2024
commencement_eligible = pd.read_excel("degree_status.xlsx", header = 1, dtype = str)

In [ ]:
commencement_eligible.head()

In [ ]:
# Rename columns for merging consistency
commencement_eligible.rename(columns={'SID': 'ID'}, inplace=True)

commencement_eligible.head()



In [ ]:
# Remove duplicates in 'commencement_eligible' based on 'ID'
commencement_eligible = commencement_eligible.drop_duplicates(subset='ID', keep='first')

In [ ]:
# Peform another merge based on ID, here we need expected graduation and cumulative GPA
df_to_extract = pd.merge(df_cleaned, commencement_eligible[['ID', 'Expected Graduation Term', 'Cumulative GPA']], on='ID', how='left')

In [ ]:
# Get all students eligible for graduation
non_empty_grad_terms = df_to_extract[
    df_to_extract['Expected Graduation Term'].notna() |
    df_to_extract['Notes'].str.contains('ready', case=False, na=False)
]

non_empty_grad_terms

In [ ]:
# Check for duplicates
duplicates = df_to_extract.duplicated(subset='ID', keep=False)

df_to_extract[duplicates]       # NO DUPLICATED DATA

In [ ]:
# Use this query for extracting all available GPAs
gpa_list = pd.read_excel("gpa_list.xlsx", header = 1)

In [ ]:
gpa_list.head()

In [ ]:
# Here we need GPAs dor the most recent term
gpa_list = gpa_list.drop_duplicates(subset='ID', keep='last')

In [ ]:
gpa_list.head()

In [ ]:
# Rename columns for merging consistency
gpa_list.rename(columns={'GPA': 'Cumulative GPA'}, inplace=True)

In [ ]:
updated_df_with_gpa = pd.merge(df_to_extract, gpa_list[['ID', 'Cumulative GPA']], on='ID', how='left', suffixes=('', '_from_gpalist'))

# Populate with GPA data only if the column is not empty; skip if already contains value

if 'Cumulative GPA' in updated_df_with_gpa.columns and 'Cumulative GPA_from_gpalist' in updated_df_with_gpa.columns:
    updated_df_with_gpa['Cumulative GPA'] = updated_df_with_gpa['Cumulative GPA'].fillna(updated_df_with_gpa['Cumulative GPA_from_gpalist'])
    updated_df_with_gpa.drop(columns=['Cumulative GPA_from_gpalist'], inplace=True)
elif 'Cumulative GPA_from_gpalist' in updated_df_with_gpa.columns:
    updated_df_with_gpa.rename(columns={'Cumulative GPA_from_gpalist': 'Cumulative GPA'}, inplace=True)



In [ ]:
updated_df_with_gpa.head()

In [ ]:
# Getting total number of students with no GPA information
na_cumulative_gpa_count = updated_df_with_gpa['Cumulative GPA'].isna().sum()
print(f"Number of rows where 'Cumulative GPA' is NaN: {na_cumulative_gpa_count} out of a total of {len(updated_df_with_gpa)}.")

In [ ]:
# Final rearrangement of columns before saving
cl_sequence_upd = ['First Enrollment Term', 'ID', 'Name', 'School', 'Prim Prog', 'Item Description', 'Cumulative GPA', 'Alumni?', 'Expected Graduation Term', 'Compl Term', 'Advisor', 'Combined Email', 'Phone', 'Notes']
updated_df_with_gpa = updated_df_with_gpa[cl_sequence_upd]

In [ ]:
updated_df_with_gpa.head()

In [ ]:
# Get percentage distribution for Item Description values
value_counts = updated_df_with_gpa['Item Description'].value_counts()
value_percentages = (value_counts / len(updated_df_with_gpa)) * 100

In [ ]:
value_percentages

In [ ]:
# Save the dataset
updated_df_with_gpa.to_excel("results.xlsx", index = False)